# Predicting Height of Childrens - TASK

In [1]:
#importing necessary library and packages

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import ast


In [2]:

# Reading the input file using pandas and extracting input features and height
df = pd.read_excel('height_and_pose.xlsx')
heights = df['Height(cm)'].to_list()
in_features = df['Pose'].to_dict()


In [3]:
X_train = []
for i in range(len(in_features)):
    test = in_features[i]
    temp_dict = ast.literal_eval(test)
    store = []
    store.append(temp_dict[0]['key_points_coordinate'][0]['nose']['y'])
    store.append(temp_dict[0]['key_points_coordinate'][5]['left_shoulder']['y'])
    store.append(temp_dict[0]['key_points_coordinate'][6]['right_shoulder']['y'])
    store.append(temp_dict[0]['key_points_coordinate'][11]['left_hip']['y'])
    store.append(temp_dict[0]['key_points_coordinate'][12]['right_hip']['y'])
    store.append(temp_dict[0]['key_points_coordinate'][13]['left_knee']['y'])
    store.append(temp_dict[0]['key_points_coordinate'][14]['right_knee']['y'])
    store.append(temp_dict[0]['key_points_coordinate'][15]['left_ankle']['y'])
    store.append(temp_dict[0]['key_points_coordinate'][16]['right_ankle']['y'])
    X_train.append(store)


In [4]:
X_train = torch.Tensor(X_train)
y_train = torch.Tensor(heights)

In [5]:
# Normalizing the input data
mean = X_train.mean(dim=0)
std = X_train.std(dim=0)
X_train = (X_train - mean) / std

In [6]:
# Creating a neural network with 4 layers (9 inputs, 32-64-32 hidden, 1 output)
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        
        self.fc1 = nn.Linear(9, 32)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        
        return x

model = NeuralNetwork()

In [7]:

# Training the neural network
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)

batch_size = 32
dataset = TensorDataset(X_train, y_train)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
num_epochs = 500

In [8]:
for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(data_loader):
        
        outputs = model(inputs)
        loss = criterion(outputs, targets.unsqueeze(1))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) %100 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [100/500], Loss: 464.0170
Epoch [200/500], Loss: 883.8150
Epoch [300/500], Loss: 472.9088
Epoch [400/500], Loss: 168.9630
Epoch [500/500], Loss: 41.1076


In [9]:
# Set the model to evaluation mode
model.eval()

# Generate predictions for the input data
with torch.no_grad():
    predictions = model(X_train)

# Print the predicted height of each child in the training dataset
for i in range(len(predictions)):
    print("Child {}: Predicted Height = {:.2f} cm".format(i+1, predictions[i].item()))




Child 1: Predicted Height = 72.83 cm
Child 2: Predicted Height = 64.23 cm
Child 3: Predicted Height = 65.75 cm
Child 4: Predicted Height = 88.87 cm
Child 5: Predicted Height = 63.97 cm
Child 6: Predicted Height = 88.16 cm
Child 7: Predicted Height = 96.06 cm
Child 8: Predicted Height = 90.60 cm
Child 9: Predicted Height = 95.39 cm
